## Классификация текста с использованием CNN
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1U3vnZeD8aiDg5Gh-SjnEyJyfrTHSRTkB)

Используя данные отзывов IMDB, построим CNN для классификации документов на позитивный и негативный классы.

Источник изложения: https://github.com/bentrevett/pytorch-sentiment-analysis

В предположении, что PyTorch уже установлен, поставим дополнительные модули и загрузим модель для токенизации:

In [ ]:
# !pip3 install https://download.pytorch.org/whl/cpu/torch-1.0.1.post2-cp36-cp36m-linux_x86_64.whl

In [ ]:
!pip install torchvision

In [ ]:
!pip install torchtext

In [ ]:
# !pip3 install spacy

In [ ]:
#!python3.6 -m spacy download en
# !python3 -m spacy download en_core_web_sm

In [ ]:
# import spacy
#import en
# en_nlp = spacy.load('en_core_web_sm')

Загрузим датасет и получим из него выборку:

In [ ]:
import torch

In [ ]:
print(torch.__version__)

1.9.0+cu102


In [ ]:
SEED = 0
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

### Данные

In [ ]:
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=17uuANm7Q1CunXHfTaF7IRY9Vy7qPl5_L' -O imdb.csv

--2021-09-23 15:13:32--  https://drive.google.com/uc?export=download&id=17uuANm7Q1CunXHfTaF7IRY9Vy7qPl5_L
Resolving drive.google.com (drive.google.com)... 142.250.141.138, 142.250.141.101, 142.250.141.139, ...
Connecting to drive.google.com (drive.google.com)|142.250.141.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0c-44-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/97tedgeniapi5tv2gq86vb2pt7vudn5p/1632409950000/13414369628864094336/*/17uuANm7Q1CunXHfTaF7IRY9Vy7qPl5_L?e=download [following]
--2021-09-23 15:13:33--  https://doc-0c-44-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/97tedgeniapi5tv2gq86vb2pt7vudn5p/1632409950000/13414369628864094336/*/17uuANm7Q1CunXHfTaF7IRY9Vy7qPl5_L?e=download
Resolving doc-0c-44-docs.googleusercontent.com (doc-0c-44-docs.googleusercontent.com)... 142.251.2.132, 2607:f8b0:4023:c0d::84
Connecting to doc-0c-44-docs.googleusercontent.com (doc

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('imdb.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


### Записываем данные в удобные структуры

In [ ]:
from torchtext.legacy import data

In [ ]:
# классы Field и LabelField отвечают за то, как данные будут храниться и обрабатываться при считывании
TEXT = data.Field(tokenize='spacy') # spacy -- значит, токенизацию будет делать модуль 
LABEL = data.LabelField()

ds = data.TabularDataset(
  path='imdb.csv', format='csv',
  skip_header=True,
  fields=[('text', TEXT),
        ('label', LABEL)]
)

ds - dataset - хранит итераторы по нашим данным и меткам класса.

**NB**: неважно как столбцы назывались в исходном датасете, здесь за названия отвечают строки, которые мы передали аргументу `fields`.



In [ ]:
next(ds.text)[:10]

['One',
 'of',
 'the',
 'other',
 'reviewers',
 'has',
 'mentioned',
 'that',
 'after',
 'watching']

In [ ]:
next(ds.label)

'positive'

Строим словарь и загружаем эмбеддинги.

С учётом того, что в коллекции 100К уникальных слов, и векторы получатся достаточно громоздкие, урежем коллекцию до 25К слов, для всех прочих заведя токен unk (unknown).

У torchtext есть репозиторий, где хранятся некоторые словарные эмбеддинги для английского. `vectors="glove.6B.100d"` значит, что крооме построения индекса слов корпуса, мы скачаем и сохраним вектора glove из этого репозитория.

In [ ]:
TEXT.build_vocab(ds, max_size=25000, vectors="glove.6B.100d")
LABEL.build_vocab(ds)

.vector_cache/glove.6B.zip: 862MB [02:40, 5.36MB/s]                           
100%|█████████▉| 399999/400000 [00:14<00:00, 27750.91it/s]


In [ ]:
# itos == i to s == index to string
print(TEXT.vocab.itos[:20])

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is', 'in', 'I', 'it', 'that', '"', "'s", 'this', '-', '/><br', 'was']


In [ ]:
TEXT.vocab.itos[:20]

['<unk>',
 '<pad>',
 'the',
 ',',
 '.',
 'a',
 'and',
 'of',
 'to',
 'is',
 'in',
 'I',
 'it',
 'that',
 '"',
 "'s",
 'this',
 '-',
 '/><br',
 'was']

In [ ]:
# stoi == s to i == string to index
TEXT.vocab.stoi[42]

0

Разобьём обучающий сет на обучение, валидацию для настройки параметров и тест.

In [ ]:
train, val = ds.split() # дефолтное соотношение 0.7
val, test = val.split(split_ratio=0.5)

In [ ]:
print(len(train))
print(len(val))
print(len(test))

35000
7500
7500


А теперь создадим итераторы батчей:

In [ ]:
BATCH_SIZE  = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, val, test), 
    batch_size=BATCH_SIZE, 
    sort=True,
    sort_key=lambda x: len(x.text), # сорируем тексты по длине, чтобы рядом оказывались предложения с одинаковой длиной и добавлялось меньше паддинга
    repeat=False)

Заглянем внутрь батча

In [ ]:
for i, batch in enumerate(test_iterator):
  print(batch.batch_size)
  # pass

64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
12


In [ ]:
batch.fields

dict_keys(['text', 'label'])

In [ ]:
batch.batch_size

12

In [ ]:
batch.text

tensor([[  149,   455,    50,  ...,    11,  2823,   399],
        [  149,     0,    25,  ...,   461,     0, 23467],
        [  149,     0,  9509,  ...,  3791,   108,    23],
        ...,
        [    2,     1,     1,  ...,     1,     1,     1],
        [ 5352,     1,     1,  ...,     1,     1,     1],
        [    4,     1,     1,  ...,     1,     1,     1]])

In [ ]:
batch.label

tensor([0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1])

## Обучение

### Модель

In [ ]:
import torch.nn as nn

Для создания свёрточного слоя воспользуемся nn.Conv2d, in_channels в нашем случае один (текст), out_channels -- это число число фильтров и размер ядер всех фильтров. Каждый фильтр будет иметь размерность [n x размерность эмбеддинга], где n - размер обрабатываемой n-граммы.

Важно, что предложения имели длину не меньше размера самого большого из используемых фильтров (здесь это не страшно, поскольку в используемых данных нет текстов, состоящих из пяти и менее слов).

In [ ]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout_proba):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.conv_0 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[0], embedding_dim))
        self.conv_1 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[1], embedding_dim))
        self.conv_2 = nn.Conv2d(in_channels=1, out_channels=n_filters, kernel_size=(filter_sizes[2], embedding_dim))
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout_proba)
        
    def forward(self, x):
        #x = [sent len, batch size]
        x = x.permute(1, 0)
                
        #x = [batch size, sent len]
        embedded = self.embedding(x)
        #print(embedded.shape)
                
        #embedded = [batch size, sent len, emb dim]
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        conv_0 = self.conv_0(embedded)
        #print(conv_0.shape)
        conv_0 = conv_0.squeeze(3)
        #print(conv_0.shape)
        conved_0 = F.relu(conv_0)
        conved_1 = F.relu(self.conv_1(embedded).squeeze(3))
        conved_2 = F.relu(self.conv_2(embedded).squeeze(3))
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        #print(conved_0.shape)
        pool_0 = F.max_pool1d(conved_0, conved_0.shape[2])
        #print(pool_0.shape)

        pooled_0 = pool_0.squeeze(2)
        #print(pooled_0.shape)
        pooled_1 = F.max_pool1d(conved_1, conved_1.shape[2]).squeeze(2)
        pooled_2 = F.max_pool1d(conved_2, conved_2.shape[2]).squeeze(2)
        
        #pooled_n = [batch size, n_filters]
        cat = self.dropout(torch.cat((pooled_0, pooled_1, pooled_2), dim=1))

        #cat = [batch size, n_filters * len(filter_sizes)]
        return self.fc(cat)

Сейчас мы можем использовать только три различных фильтра, хотелось бы больше. Вообще, можно воспользоваться `nn.ModuleList`, чтобы создать слои списком и сделать так, чтобы фильтров создавалось по количеству элементов в filter_sizes. [(Как здесь).](https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/4%20-%20Convolutional%20Sentiment%20Analysis.ipynb)

### Вспомогательные функции

Опишем функцию подсчёта accuracy, а также функции обучения и применения сети:

In [ ]:
import torch.nn.functional as F

def binary_accuracy(preds, y):
    rounded_preds = torch.round(F.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train_func(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        
        predictions = model(batch.text.cuda()).squeeze(1)

        loss = criterion(predictions.float(), batch.label.float().cuda())
        acc = binary_accuracy(predictions.float(), batch.label.float().cuda())
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss
        epoch_acc += acc
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate_func(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text.cuda()).squeeze(1)

            loss = criterion(predictions.float(), batch.label.float().cuda())
            acc = binary_accuracy(predictions.float(), batch.label.float().cuda())

            epoch_loss += loss
            epoch_acc += acc
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

### Подготовка обучения

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT_PROBA = 0.5

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT_PROBA)

In [ ]:
model # посмотрим на неё ещё раз

CNN(
  (embedding): Embedding(25002, 100)
  (conv_0): Conv2d(1, 100, kernel_size=(3, 100), stride=(1, 1))
  (conv_1): Conv2d(1, 100, kernel_size=(4, 100), stride=(1, 1))
  (conv_2): Conv2d(1, 100, kernel_size=(5, 100), stride=(1, 1))
  (fc): Linear(in_features=300, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

Копируем скачанные эмбеддинги слов в параметры слоя `Embedding`, чботы не нужно было обучать его с нуля.

In [ ]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.4413,  0.3325,  0.1120,  ..., -0.0686,  0.4374,  0.8717],
        [ 0.1177,  0.1141,  0.2218,  ..., -1.0694,  0.4712, -0.7554],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [ ]:
import torch.optim as optim

In [ ]:
optimizer = optim.Adam(model.parameters()) # мы подали оптимизатору все параметры -- значит, эмбеддиги тоже будут дообучаться
criterion = nn.BCEWithLogitsLoss() # бинарная кросс-энтропия с логитами

model = model.cuda() # будем учить на gpu! =)

### Обучение!

Используя определённые ранее функции, запустим обучение с оптимизатором Adam и оценим качество на валидации и тесте:

In [ ]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):
    train_loss, train_acc = train_func(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate_func(model, valid_iterator, criterion)
    
    print(f'Epoch: {epoch+1:02}, Train Loss: {train_loss:.3f}, Train Acc: {train_acc*100:.2f}%, Val. Loss: {valid_loss:.3f}, Val. Acc: {valid_acc*100:.2f}%')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 01, Train Loss: 0.406, Train Acc: 81.15%, Val. Loss: 0.298, Val. Acc: 87.31%
Epoch: 02, Train Loss: 0.252, Train Acc: 89.65%, Val. Loss: 0.253, Val. Acc: 89.87%
Epoch: 03, Train Loss: 0.180, Train Acc: 93.05%, Val. Loss: 0.239, Val. Acc: 90.65%


In [ ]:
test_loss , test_acc = evaluate_func(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f}, Test Acc: {test_acc*100:.2f}%')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1805: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Test Loss: 0.282, Test Acc: 88.24%


#### Упражнение 1: как изменились эмбеддинги?

Давайте проверим, произошли ли какие-то любопытные изменения в отношениях между словами.

In [ ]:
TEXT.vocab.vectors # старые эмбеддинги

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.4413,  0.3325,  0.1120,  ..., -0.0686,  0.4374,  0.8717],
        [ 0.1177,  0.1141,  0.2218,  ..., -1.0694,  0.4712, -0.7554],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [ ]:
model.embedding.weight.data # новые эмбеддиги

tensor([[ 1.3537, -0.5994, -0.2251,  ...,  0.0064, -1.3333,  0.3274],
        [-0.2687, -0.0785, -1.9346,  ..., -1.7024, -0.1569,  0.8511],
        [ 0.4343, -0.0671, -1.0944,  ..., -0.3915, -0.9669,  0.3443],
        ...,
        [-0.5355,  1.5282,  0.3985,  ...,  0.6649, -1.7519, -1.8649],
        [ 0.2111,  1.9653,  1.2217,  ..., -1.6369,  0.9221,  0.5238],
        [-1.5200,  0.1436,  1.1655,  ...,  0.0534,  0.7600, -1.1566]],
       device='cuda:0')

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
i1, i2 = TEXT.vocab.stoi['perfect'], TEXT.vocab.stoi['awful']

In [ ]:
cosine_similarity([
  TEXT.vocab.vectors[i1].cpu().numpy(),
  TEXT.vocab.vectors[i2].cpu().numpy()
  ])

array([[0.9999999, 0.5248411],
       [0.5248411, 0.9999996]], dtype=float32)

In [ ]:
cosine_similarity([
  model.embedding.weight.data[i1].cpu().numpy(),
  model.embedding.weight.data[i2].cpu().numpy()
  ])

array([[ 0.9999999, -0.086183 ],
       [-0.086183 ,  0.9999996]], dtype=float32)

"прекрасный" и "ужасный" стали дальше друг от друга

**Задание**: посмотрите на другие изменения и попробуйте их объяснить. Для наглядности можно сделать визуализацию с помощью t-sne.

#### Упражнение 2: nn.ModuleList

Используя его, можно легко и красиво определить столько разных сверток, сколько захотим! Вот пример:

In [ ]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
                
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
                
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            

**Задание**: поэкспериментируйте с количеством и размером сверток. Что сработает лучше?

#### Упражнение 3: другая предобработка

При загрузке данных, мы использовали `data.Field(tokenize='spacy')`.
Попробуем заменить токенизатор `spacy` на свою функцию, которая дополнительно чистит данные от мусора.

In [ ]:
# пример мусора
ds.examples[0].text[25:40]

['is',
 'exactly',
 'what',
 'happened',
 'with',
 'me.<br',
 '/><br',
 '/>The',
 'first',
 'thing',
 'that',
 'struck',
 'me',
 'about',
 'Oz']

Предобработка (из прошлого семинара):

In [ ]:
from bs4 import BeautifulSoup
import re

In [ ]:
def review_to_wordlist(review):
    # убираем ссылки
    review = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", " ", review)
    # достаем сам текст
    review_text = BeautifulSoup(review, "lxml").get_text()
    # оставляем только буквенные символы
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    # приводим к нижнему регистру и разбиваем на слова по символу пробела
    return review_text.lower().split() 

**Задание**: попробуйте обучить модель, используя другую предобработку. Стало ли лучше? Что если убирать стоп-слова?

# Аугментация данных

В нашем примере данные были сбалансированными, а как работать с небалансированными данными?

Рассмотрим задачу распознавания тональности твитов, взятых из [Twitter Sentimental Analysis challenge](https://datahack.analyticsvidhya.com/contest/practice-problem-twitter-sentiment-analysis/).

Источник изложения: https://github.com/mabusalah/Resampling

Получим данные

In [ ]:
!wget --no-check-certificate "https://drive.google.com/uc?export=download&id=1Jjuk23nMTQkfA3-3_HpevXGeupav7QLz" -O train.csv
!wget --no-check-certificate "https://drive.google.com/uc?export=download&id=11FugxTRrdKqkDE_3KlfCDWRn_rbR6VxM" -O test.csv

--2021-09-23 14:47:37--  https://drive.google.com/uc?export=download&id=1Jjuk23nMTQkfA3-3_HpevXGeupav7QLz
Resolving drive.google.com (drive.google.com)... 142.251.2.139, 142.251.2.102, 142.251.2.113, ...
Connecting to drive.google.com (drive.google.com)|142.251.2.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-44-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7u8rheh5adhdiu2fhv5a3r2n5r6siiss/1632408450000/13414369628864094336/*/1Jjuk23nMTQkfA3-3_HpevXGeupav7QLz?e=download [following]
--2021-09-23 14:47:38--  https://doc-0s-44-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/7u8rheh5adhdiu2fhv5a3r2n5r6siiss/1632408450000/13414369628864094336/*/1Jjuk23nMTQkfA3-3_HpevXGeupav7QLz?e=download
Resolving doc-0s-44-docs.googleusercontent.com (doc-0s-44-docs.googleusercontent.com)... 142.251.2.132, 2607:f8b0:4023:c0d::84
Connecting to doc-0s-44-docs.googleusercontent.com (doc-0s-44-d

In [ ]:
import pandas as pd
test = pd.read_csv('test.csv')
print("Test Set:"% test.columns, test.shape, len(test))
train = pd.read_csv('train.csv')
print("Training Set:"% train.columns, train.shape, len(train))

Test Set: (17197, 2) 17197
Training Set: (31962, 3) 31962


In [ ]:
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [ ]:
test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


Итак, посмотрим, какой процент от общей выборки занимают позитивные и негативные примеры.

In [ ]:
print("Positive: ", train.label.value_counts()[0]/len(train)*100,"%")
print("Negative: ", train.label.value_counts()[1]/len(train)*100,"%")

Positive:  92.98542018647143 %
Negative:  7.014579813528565 %


93% vs. 7% - данные определенно несбалансированны, что, в свою очередь, негативно влияет на точность предсказания.
Для начала поработаем с исходными данными и оценим точность классификации.
Начнем с предобработки данных: уберем из твитов числа, html/xml-тэги, специальные символы.

In [ ]:
import re
from bs4 import BeautifulSoup #для работы с html/xml-тэгами
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import PorterStemmer

porter=PorterStemmer()
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

def tweet_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()

    words = tok.tokenize(lower_case)
    
    stem_sentence=[]
    for word in words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    words="".join(stem_sentence).strip()
    return words

nums = [0,len(train)]
clean_tweet_texts = []
for i in range(nums[0],nums[1]):
    clean_tweet_texts.append(tweet_cleaner(train['tweet'][i]))
    
nums = [0,len(test)]
test_tweet_texts = []

for i in range(nums[0],nums[1]):
    test_tweet_texts.append(tweet_cleaner(test['tweet'][i])) 
    
train_clean = pd.DataFrame(clean_tweet_texts,columns=['tweet'])
train_clean['label'] = train.label
train_clean['id'] = train.id
test_clean = pd.DataFrame(test_tweet_texts,columns=['tweet'])
test_clean['id'] = test.id

Разделим данные на обучающие и проверочные.

In [ ]:
from sklearn import model_selection, preprocessing, metrics, linear_model, svm

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(train_clean['tweet'],train_clean['label'])
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

Рассчитаем TF-IDF признаки.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=100000)
tfidf_vect.fit(train_clean['tweet'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

Точность в качестве метрики работает хорошо только на сбалансированных наборах данных, поэтому для оценки результатов работы  алгоритма будем использовать F1-метрику.

In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    classifier.fit(feature_vector_train, label)

    predictions = classifier.predict(feature_vector_valid)    

    return metrics.f1_score(valid_y,predictions)

Для начала обучим лог-регрессию.

In [ ]:
accuracyORIGINAL = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),xtrain_tfidf, train_y, xvalid_tfidf)
print ("Logistic regression Baseline, WordLevel TFIDF: ", accuracyORIGINAL)

Logistic regression Baseline, WordLevel TFIDF:  0.5401459854014599


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Попробуйте использовать обычный счетчик слов для извлечения признаков.

Как видно, результат оставляет желать лучшего.

Что можно сделать с данными?

Было бы неплохо как-то увеличить  количество негативных примеров, или же уменьшить количество положительных. Для этого существуют различные техники аугментации данных. 
В Python для этих целей есть библиотека imblearn (imbalanced-learn).

In [ ]:
from imblearn.over_sampling import BorderlineSMOTE, SMOTE, ADASYN, SMOTENC, RandomOverSampler
from imblearn.under_sampling import (RandomUnderSampler, 
                                    NearMiss, 
                                    InstanceHardnessThreshold,
                                    CondensedNearestNeighbour,
                                    EditedNearestNeighbours,
                                    RepeatedEditedNearestNeighbours,
                                    AllKNN,
                                    NeighbourhoodCleaningRule,
                                    OneSidedSelection,
                                    TomekLinks)
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.pipeline import make_pipeline

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


Итак, в качестве инструментов для аугментации рассмотрим: under-sampling, over-sampling и их комбинацию.

**Under-sampling** уравновешивает данные за счет уменьшения размера  превалирующего класса.
Этот метод разумно использовать, когда количество данных достаточно велико, иначе есть риск остаться и вовсе без обучающих примеров.

Итак, логика действия довольно проста: мы просто случайным образом убираем лишние экземпляры из превалирующего класса.

Так как в нашем примере лишь 7% всех твитов имеют негативную окраску, уравновешивание позитивного набора с этими 7-ю процентами вряд ли обеспечит хороший результат.

Попробуем...

In [ ]:
rus = RandomUnderSampler(random_state=0, replacement=True)
rus_xtrain_tfidf, rus_train_y = rus.fit_sample(xtrain_tfidf, train_y)
accuracyrus = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),rus_xtrain_tfidf, rus_train_y, xvalid_tfidf)
print ("Logistic regressio RUS, WordLevel TFIDF: ", accuracyrus)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Logistic regressio RUS, WordLevel TFIDF:  0.5184033177812338


Действительно, все стало только хуже.

Попробуем другие алгоритмы **under-sampling**.

Например, **NearMiss**. Данный алгоритм выбирает, какие экземпляры нужно оставить в превалирующем классе на основании некоторых эвристик. Существует три варианта данного алгоритма:

**NearMiss-1** оставляет те экземпляры из превалирующего класса, для которых среднее расстояние до *k* ближайших соседей из миноритарного класса будет наименьшим.

**NearMiss-2** оставляет те экземпляры из превалирующего класса, для которых среднее расстояние до *k* самых дальних соседей из миноритарного класса будет наименьшим.

**NearMiss-3** состоит из двух шагов: сначала, для каждого экземпляра из миноритарного класса выбирается *k* ближайших соседей из превалирующего класса, затем, из большего класса выбираются те экземпляры, для которых среднее расстояние до *k* ближайших соседей максимальное.

![](https://glemaitre.github.io/imbalanced-learn/_images/sphx_glr_plot_nearmiss_001.png)

In [ ]:
for sampler in (NearMiss(version=1),NearMiss(version=2),NearMiss(version=3)):
    nm_xtrain_tfidf, nm_train_y = sampler.fit_sample(xtrain_tfidf, train_y)
    accuracysm = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),nm_xtrain_tfidf, nm_train_y, xvalid_tfidf)
    print ("Logistic regression NearMiss(version= {0}), WordLevel TFIDF: ".format(sampler.version), accuracysm)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Logistic regression NearMiss(version= 1), WordLevel TFIDF:  0.3019441069258809


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Logistic regression NearMiss(version= 2), WordLevel TFIDF:  0.5128205128205128


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:194: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn('The number of the samples to be selected is larger'


Logistic regression NearMiss(version= 3), WordLevel TFIDF:  0.3129129129129129


**Edited Nearest Neighbor (ENN)**

ENN удаляет из большего класса элемент, если класс его ближайшего соседа отличается от его собственного.

In [ ]:
enn_xtrain_tfidf, enn_train_y = EditedNearestNeighbours().fit_sample(xtrain_tfidf, train_y)
accuracy = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),enn_xtrain_tfidf, enn_train_y, xvalid_tfidf)
print ("Logistic regression {0}, WordLevel TFIDF: ", accuracy)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Logistic regression {0}, WordLevel TFIDF:  0.5518072289156627


Как вы поняли, при применении **Under-samplin**g техник новые данные не генерируются, в отличие от **Over-sampling**.

# Over-sampling

Итак, когда данных недостаточно или количество экземпляров в миноритарном классе очень мало применяется **Over-sampling**. 

При применении этой техники балансировка данных происходит за счет увеличения количества экземпляров в миноритарном классе. Новые элементы генерируются за счет: повторения, бутстрэппинга, SMOTE (Synthetic Minority Over-Sampling Technique) или ADASYN (Adaptive synthetic sampling).

**Random Over-sampling**: случайным образом дублируются некоторые элементы из миноритарного класса.

In [ ]:
#Random Over Sampling
ros = RandomOverSampler(random_state=777)
ros_xtrain_tfidf, ros_train_y = ros.fit_sample(xtrain_tfidf, train_y)
accuracyROS = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),ros_xtrain_tfidf, ros_train_y, xvalid_tfidf)
print ("Logistic regression ROS, WordLevel TFIDF: ", accuracyROS)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Logistic regression ROS, WordLevel TFIDF:  0.6687354538401862


**SMOTE Over-sampling**

Алгоритм SMOTE основан на идее генерации некоторого количества искусственных примеров, которые были бы «похожи» на имеющиеся в миноритарном классе, но при этом не дублировали их.

Для создания новой записи находят разность $d=X_b-X_a,$ где $ X_b, X_a -$ векторы признаков «соседних» примеров $a$ и $b$ из миноритарного класса. 

Их находят, используя алгоритм ближайшего соседа (*KNN*). В данном случае необходимо и достаточно для примера $b$ получить набор из $k$ соседей, из которого в дальнейшем будет выбрана запись $a$. Остальные шаги алгоритма *KNN* не требуются.

Далее из $d$ путем умножения каждого его элемента на случайное число в интервале (0, 1) получают $\hat{d}$. Вектор признаков нового примера вычисляется путем сложения $X_a$ и $\hat{d}$. 

Алгоритм **SMOTE** позволяет задавать количество записей, которое необходимо искусственно сгенерировать. Степень сходства примеров $a$ и $b$ можно регулировать путем изменения значения $k$ (числа ближайших соседей).

![](https://hsto.org/getpro/habr/post_images/c57/e7e/f4f/c57e7ef4f8711ad2eda881651a027867.png)

In [ ]:
sm = SMOTE(random_state=777, ratio = 1.0)
sm_xtrain_tfidf, sm_train_y = sm.fit_sample(xtrain_tfidf, train_y)
accuracySMOTE = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),sm_xtrain_tfidf, sm_train_y, xvalid_tfidf)
print ("Logistic regression SMOTE, WordLevel TFIDF: ", accuracySMOTE)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Logistic regression SMOTE, WordLevel TFIDF:  0.6593233674272228


Итак, по сравнению с **Random Over-sampling** разница небольшая.

Проверьте результаты **Random Over-sampling** и **SMOTE Over-sampling** для реальных тестовых данных (*test_clean*).

Следующий алгоритм **ASMO: Adaptive synthetic minority oversampling**.



Сгенерировать искусственные записи в пределах отдельных кластеров на основе всех классов. Для каждого примера миноритарного класса находят m ближайших соседей, и на основе них (также как в SMOTE) создаются новые записи.

1.   Если для каждого $i$-ого примера миноритарного класса из $k$ ближайших соседей $g$ ($g\leq k$) принадлежит к мажоритарному, то набор данных считается «рассеянным». В этом случае используют алгоритм **ASMO**, иначе применяют **SMOTE** (как правило, $g$ задают равным 20).
2.   Используя только примеры миноритарного класса, выделить несколько кластеров (например, алгоритмом $k$-means).
3.   Сгенерировать искусственные записи в пределах отдельных кластеров на основе всех классов. Для каждого примера миноритарного класса находят m ближайших соседей, и на основе них (также как в **SMOTE**) создаются новые записи.

![](https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQdTzjHBZ_9At5GIDRpF2AAw9hU1jzcVE5uwA&usqp=CAU)

Такая модификация алгоритма **SMOTE** делает его более адаптивным к различным наборам данных с несбалансированными классами.

In [ ]:
ad = ADASYN(random_state=777, ratio = 1.0)
ad_xtrain_tfidf, ad_train_y = ad.fit_sample(xtrain_tfidf, train_y)
accuracyADASYN = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),ad_xtrain_tfidf, ad_train_y, xvalid_tfidf)
print ("Logistic regression ADASYN, WordLevel TFIDF: ", accuracyADASYN)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Logistic regression ADASYN, WordLevel TFIDF:  0.6555639666919001


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


И опять проверим на реальных тестовых примерах.

# Комбинация **Under-** и **Over-sampling**

В *imblearn* реализованы две возможные комбинации:


1.   **SMOTE** + **ENN**
2.   **SMOTE** + **Tomek Link Removal** (Пара двух ближайших соседей, которые принадлежат разным классам называется *Tomek link*. Under-sampling заключается в удалении всех таких элементов из мажоритарного класса)

Подробнее: https://imbalanced-learn.readthedocs.io/en/stable/api.html#module-imblearn.combine



In [ ]:
se = SMOTEENN(random_state=42)
se_xtrain_tfidf, se_train_y = se.fit_sample(xtrain_tfidf, train_y)
accuracy = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),se_xtrain_tfidf, se_train_y, xvalid_tfidf)
print ("Logistic regression SMOTEENN: ", accuracy)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Logistic regression SMOTEENN:  0.5139720558882237


Первый метод сработал плохо. Оцените работу второго подхода.

# Аугментация текстовых данных

Для аугментации тектсовых данных будем применять библиотеку [nlpaug](https://github.com/makcedward/nlpaug).

Библиотека предоставляет функционал для различных типов аугментации текста, в том числе: добавление опечаток, замена слов синонимами, вставка дополнительных слов и т.д. 

In [ ]:
!pip install numpy requests nlpaug

In [ ]:
text = 'The quick brown fox jumps over the lazy dog .'
print(text)

The quick brown fox jumps over the lazy dog .


Аугментируем пример с помощью замены слов на основе векторной модели предсавления слов Glove.

Загрузим и инициализируем модель:

In [ ]:
from nlpaug.util.file.download import DownloadUtil


DownloadUtil.download_glove(model_name='glove.6B', dest_dir='.')

In [ ]:
import nlpaug.augmenter.word as naw

aug = naw.WordEmbsAug(
    model_type='glove', model_path='glove.6B.100d.txt',
    action="substitute")

Аугментируем наш пример:

In [ ]:
augmented_text = aug.augment(text)
print("Original:")
print(text)
print("Augmented Text:")
print(augmented_text)

Original:
The quick brown fox jumps over the lazy dog .
Augmented Text:
The quick brown shown turns over the lazy monster.


Попробуем аугментировать негативные твиты для классификатора. 

In [ ]:
def get_negative_tweets(train_x, train_y):
  x_neg_train = []
  for tweet, label in zip(train_x, train_y):
    if label == 1:
      x_neg_train.append(tweet)
  return x_neg_train

In [ ]:
neg_train_x = get_negative_tweets(train_x, train_y)
neg_train_x[:10]

['um whi a girl and not a boy or mayb just gender neutral teenag',
 'clearli just humour attempt by liber media to undermin trump racebait won t work trump is not a',
 'w prop jesu wa not black nor wa he white those who promot the hate biggotri need to be call vile scum',
 'thi moron is lead the us down a path that ha so much potenti to damag and destroy the world',
 'techjunkiejh the daili show unpack against blackwomen and asianmen on datingapp',
 'you might be a libtard if libtard sjw liber polit',
 'you might be a libtard if libtard sjw liber polit',
 'allahsoil not all muslim hate america emirati in word',
 'if they say their comment are just a joke or smth it will be just their opinion i don t think so it s a',
 'riyadh is renown for some of the deadliest traffic in the world in word']

In [ ]:
neg_train_x_aug = aug.augment(neg_train_x)
neg_train_x_aug[:10]

['um whi to her and not a boy or mayb just feminism extremely teenag',
 'clearli just humour attempt by jahrbuch media up undermin trump racebait trophy t did trump actually not a',
 "w prop jesu wa believe black unless wa he white those thought promot the hate biggotri n't to one call awful scum",
 'thi arka where lead the coming down a turn that ha so much potenti out damag and destroy the record',
 'techjunkiejh another daili show individualize against blackwomen way asianmen on datingapp',
 'tell might be a libtard if libtard js03 poupée polit',
 'feel might be long libtard if libtard shn liber polit',
 'allahsoil not it muslim remember america emirati in word',
 'if they say their comment are just it joke or smth its time be just up opinion mind don t think so you price a',
 'riyadh is admired for some an the deadliest traffic with even world in word']

Добавим аугментированные данные к обучающей выборке:

In [ ]:
train_x_aug = train_x.append(pd.Series(neg_train_x_aug))

In [ ]:
import numpy as np

neg_train_y_aug = [1] * len(neg_train_x_aug)
train_y_aug = np.concatenate((train_y, neg_train_y_aug), axis=0)

Перемешаем обучающие данные:

In [ ]:
import random
from tqdm import tqdm

indexes = list(range(0, len(train_x_aug)))
random.shuffle(indexes)

train_x_aug_shuffled = pd.Series([list(train_x_aug)[i] for i in tqdm(indexes)])
train_y_aug_shuffled = train_y_aug[indexes]

100%|██████████| 25640/25640 [00:56<00:00, 456.65it/s]


In [ ]:
#tweets = pd.concat([train_x1, valid_x])

Векторизуем полученные тексты:

In [ ]:
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=100000)
tfidf_vect.fit(pd.concat([train_x_aug_shuffled, valid_x]))
xtrain_tfidf_aug =  tfidf_vect.transform(train_x_aug_shuffled)
xvalid_tfidf_aug =  tfidf_vect.transform(valid_x)

Обучим и оценим модель:

In [ ]:
accuracyTEXTAUG = train_model(linear_model.LogisticRegression(random_state=0, solver='lbfgs',multi_class='multinomial'),xtrain_tfidf_aug, train_y_aug_shuffled, xvalid_tfidf_aug)
print ("Logistic regression with augmented texts, WordLevel TFIDF: ", accuracyTEXTAUG)

Logistic regression with augmented texts, WordLevel TFIDF:  0.42416107382550333
